IDEA FOR LATER - "description" is useless for machine analysis in its current form, and dummy variables will be unintelligible. But what if I used sentiment analysis and assigned a score accordingly? Would have to translate first, how reliable would translations be?

In [1]:
import pandas as pd
import numpy as np
import re
import ast

In [2]:
scout_car = pd.read_json('scout_car.json', lines=True)
scout_car.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,...,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,...,"[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,...,[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,...,"[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,...,"[\nAudi A1: , - 1e eigenaar , - Perfecte staat...",NaN,[\n6\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,NaN,...,"[\n, Technik & Sicherheit:, Xenon plus, Klimaa...",NaN,NaN,[\nGermany\n],NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
scout_car.columns

Index(['url', 'make_model', 'short_description', 'body_type', 'price', 'vat',
       'km', 'registration', 'prev_owner', 'kW', 'hp', 'Type',
       'Previous Owners', 'Next Inspection', 'Inspection new', 'Warranty',
       'Full Service', 'Non-smoking Vehicle', 'null', 'Make', 'Model',
       'Offer Number', 'First Registration', 'Body Color', 'Paint Type',
       'Body Color Original', 'Upholstery', 'Body', 'Nr. of Doors',
       'Nr. of Seats', 'Model Code', 'Gearing Type', 'Displacement',
       'Cylinders', 'Weight', 'Drive chain', 'Fuel', 'Consumption',
       'CO2 Emission', 'Emission Class', '\nComfort & Convenience\n',
       '\nEntertainment & Media\n', '\nExtras\n', '\nSafety & Security\n',
       'description', 'Emission Label', 'Gears', 'Country version',
       'Electricity consumption', 'Last Service Date', 'Other Fuel Types',
       'Availability', 'Last Timing Belt Service Date', 'Available from'],
      dtype='object')

# Formatting

Convert all lists within columns to strings for processing. Delete columns with 90% or greater nulls values.

In [4]:
scout_car = scout_car.loc[:, scout_car.isnull().mean() <= 0.90]
scout_car = scout_car.map(lambda x: str(x) if isinstance(x, list) else x)

Now we will go through the columns in order.

# Cleaning

These columns have nothing wrong with them, but I will rename them for aesthetic purposes.

In [5]:
scout_car.rename(columns={'url': 'URL'}, inplace=True)
scout_car.rename(columns={'short_description': 'Short Description'}, inplace=True)
scout_car.rename(columns={'body_type': 'Body Type'}, inplace=True)
scout_car.rename(columns={'price': 'Price'}, inplace=True)
scout_car.rename(columns={'vat': 'VAT'}, inplace=True)
#scout_car.rename(columns={'km': 'KM'}, inplace=True)
#scout_car.rename(columns={'hp': 'Horsepower'}, inplace=True)

'km' is a string. It needs to be converted to an integer by stripping out the 'km'

In [6]:
def extract_km(entry):
    if not isinstance(entry, str):
        return pd.NA
    match = re.search(r'([\d,]+)', entry)
    if match:
        # Remove commas and convert to int
        return int(match.group(1).replace(',', ''))
    return pd.NA

scout_car['km'] = scout_car['km'].apply(extract_km).astype('Int64')
scout_car.rename(columns={'km': 'KM'}, inplace=True)

'hp' (Horsepower) is a string. It needs to be converted to an integer by stripping out the 'kW'

In [7]:
import re
import pandas as pd

def extract_kw(entry):
    if not isinstance(entry, str):
        return pd.NA
    match = re.search(r'(\d+)', entry)
    if match:
        return int(match.group(1))
    return pd.NA

scout_car['hp'] = scout_car['hp'].apply(extract_kw)
scout_car.rename(columns={'hp': 'Horsepower (kW)'}, inplace=True)
# Was going to convert to INT here, but wouldn't work because of null values.
# I'll deal with that in the next notebook.

"registration" has no data integrity issues, but its formatting will be problematic for numeric processing.

(HOLDING OFF ON THIS STEP UNTIL THE NEXT NOTEBOOK)
I will change the formatting from "01/2016" to "201601" so that higher numbers correspond to newer dates.


In [8]:
scout_car['registration'].replace('-/-', np.nan, inplace=True)

/var/folders/cf/4zrkk0lx1nbdx9tpklpk4jc00000gn/T/ipykernel_17137/3855305649.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  scout_car['registration'].replace('-/-', np.nan, inplace=True)


"prev_owner" contains entirely redunant data with "Previous Owners" so it can be dropped.

In [9]:
scout_car.drop('prev_owner', axis=1, inplace=True)

"Type" also contains redunant data from "Fuel", so we will parse out only the first string.

In [10]:
def get_first_nonempty(entry):
    # Skip NaN or non-string values
    if not isinstance(entry, str):
        return None
    
    # remove [ ] brackets if present
    entry = entry.strip("[]")
    # split on commas
    parts = [p.strip(" '\"") for p in entry.split(",")]
    # take the first non-empty piece
    for p in parts:
        if p:
            return p
    return None

scout_car['Type'] = scout_car['Type'].apply(get_first_nonempty)

"Previous Owners" is ready as-is.

"Next Inspection" also contains irrevelant data. We will parse out the relevant date and reformat it as we did with the registration date.

In [11]:
# Extract date, convert to YYYYMM format, and rename column.
import re
import ast
import pandas as pd

# matches 1- or 2-digit month and 4-digit year, e.g. 7/2019 or 07/2019
date_re = re.compile(r'\b(0?[1-9]|1[0-2])/[0-9]{4}\b')

def extract_first_date(val):
    # handle NaNs
    if pd.isna(val):
        return None

    # If it's already a list/tuple, check each element for a date
    if isinstance(val, (list, tuple)):
        for item in val:
            if not isinstance(item, str):
                continue
            m = date_re.search(item)
            if m:
                return m.group(0)
            s = item.strip(" []'\"\n\t")
            if s:
                return s  # if no date but contains text, return the first text
        return None

    # If it's a string, try to find a date directly (works even if there are \n)
    if isinstance(val, str):
        # try a quick regex search first (this handles "['\n07/2019\n', ...]" directly)
        m = date_re.search(val)
        if m:
            return m.group(0)

        # if no date, try to literal_eval (safe) in case it's a stringified list
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, (list, tuple)):
                for item in parsed:
                    if isinstance(item, str):
                        m = date_re.search(item)
                        if m:
                            return m.group(0)
                        s = item.strip(" []'\"\n\t")
                        if s:
                            return s
                return None
        except Exception:
            pass

        # fallback: split on common separators and return first non-empty token
        parts = re.split(r'[,\|;]', val.strip("[]"))
        for p in parts:
            p = p.strip(" '\"\n\t")
            if p:
                return p
        return None

    # other types -> no date
    return None

# Apply to DataFrame
scout_car['Next Inspection'] = scout_car['Next Inspection'].apply(extract_first_date)

# REFORMAT TO YYYYMM
# Convert to datetime
scout_car['Next Inspection'] = pd.to_datetime(
    scout_car['Next Inspection'],
    format='%m/%Y',
    errors='coerce'
)
# Convert to YYYYMM format
scout_car['Next Inspection'] = scout_car['Next Inspection'].dt.strftime('%Y%m')

# Convert to String - This is only a temporary step due to special considerations for this column.
# It will be changed back to Numeric in the next notebook.
scout_car['Next Inspection'] = scout_car['Next Inspection'].astype(str)

# Convert to INT - Will hold off on this until next notebook, because of special considerations for this column
#scout_car['Next Inspection'] = pd.to_numeric(scout_car['Next Inspection'], errors='coerce').astype('Int64')

# Remame column
scout_car.rename(columns={'Next Inspection': 'Next Inspection (YYYYMM)'}, inplace=True)

"Inspection new" I am not sure exactly what "Yes" means in this context but I will keep it. We will have to extract the first string.

In [12]:
# Extract string
import ast
import pandas as pd

def extract_first_clean_text(entry):
    if not isinstance(entry, str):
        return None
    
    try:
        # Try to parse the string as a Python list
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and len(parsed) > 0:
            first = parsed[0]
            if isinstance(first, str):
                return first.strip()
            else:
                return str(first).strip()
    except (ValueError, SyntaxError):
        # If parsing fails, fallback to manual splitting
        entry = entry.strip("[]")
        parts = [p.strip(" '\"\n\t") for p in entry.split(",")]
        for p in parts:
            if p:
                return p
    return None

scout_car['Inspection new'] = scout_car['Inspection new'].apply(extract_first_clean_text)

Warranty contains extraneous information. We will have to extract the warranty's length in months.

In [13]:
# Extract number of months and rename column
import ast
import re
import pandas as pd

def extract_warranty_months(entry):
    if not isinstance(entry, str):
        return pd.NA
    
    try:
        # Parse string to list
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and len(parsed) > 0:
            first = parsed[0]
            if isinstance(first, str):
                # Find the first number in the string
                match = re.search(r'\d+', first)
                if match:
                    return int(match.group(0))
    except (ValueError, SyntaxError):
        pass
    
    return pd.NA

scout_car['Warranty'] = scout_car['Warranty'].apply(extract_warranty_months)

# Rename column to clarify that it is measuring months
scout_car.rename(columns={'Warranty': 'Warranty (months)'}, inplace=True)

"Full service" "Non-smoking Vehicle" and "null" seem to have no useful information, so we can drop them.

In [14]:
scout_car.drop('Full Service', axis=1, inplace=True)
scout_car.drop('Non-smoking Vehicle', axis=1, inplace=True)
scout_car.drop('null', axis=1, inplace=True)

"Make" and "Model" are redundant to "make_model" but it may be useful to hav eboth separate and combined strings. We will leave them for now.

Offer number will likely be useless in a machine learning context but I'll leave it for now anyway.

In [15]:
import ast
import pandas as pd

def clean_offer_number(entry):
    if not isinstance(entry, str):
        return pd.NA
    try:
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and len(parsed) > 0:
            first = parsed[0]
            if isinstance(first, str):
                return first.strip()
    except (ValueError, SyntaxError):
        # fallback: just strip brackets and whitespace if parse fails
        return entry.strip("[] '\"\n\t")
    return pd.NA

scout_car['Offer Number'] = scout_car['Offer Number'].apply(clean_offer_number)

'First Registration' contains extraneous information along with a year.

In [16]:
import ast
import pandas as pd

def extract_first_nonempty_str(entry):
    if not isinstance(entry, str):
        return pd.NA
    try:
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)):
            for item in parsed:
                if isinstance(item, str):
                    cleaned = item.strip()
                    if cleaned:
                        return cleaned
        return pd.NA
    except (ValueError, SyntaxError):
        # fallback if not parseable
        entry = entry.strip("[] '\"\n\t")
        return entry if entry else pd.NA

scout_car['First Registration'] = scout_car['First Registration'].apply(extract_first_nonempty_str)

'Body color' can be cleaned the exact same way.

In [17]:
scout_car['Body Color'] = scout_car['Body Color'].apply(extract_first_nonempty_str)

'Paint Type' and 'Body Color Original' are also simple. We can re-use a function from earlier.

In [18]:
scout_car['Paint Type'] = scout_car['Paint Type'].apply(clean_offer_number)
scout_car['Body Color Original'] = scout_car['Body Color Original'].apply(clean_offer_number)

'Upholstery' presents a unique challenge. Each entry contains both a material and a color. We should split this column into separate columns for each.

In [19]:
scout_car['Upholstery_clean'] = (
    scout_car['Upholstery']
    .str.strip()
    .str.strip("[]")
    .str.replace(r"\\n", "", regex=True)
    .str.strip("'")   # Remove single quotes here
)

scout_car[['Upholstery Material', 'Upholstery Color']] = scout_car['Upholstery_clean'].str.split(',', n=1, expand=True)

scout_car['Upholstery Material'] = scout_car['Upholstery Material'].str.strip().str.strip("'")
scout_car['Upholstery Color'] = scout_car['Upholstery Color'].str.strip().str.strip("'")

scout_car.drop('Upholstery', axis=1, inplace=True)
scout_car.drop('Upholstery_clean', axis=1, inplace=True)

'Body' contains the same information as 'Body Type', so we can drop it.

In [20]:
scout_car.drop('Body', axis=1, inplace=True)

'Nr. of Doors' and 'Nr. of Seats' contain numbers that can be extracted relatively easily. We wlil also rename the columns to include # symbols for appearance sake.

In [21]:
# Extract numbers and rename columns
import ast
import pandas as pd

def extract_num(entry):
    if not isinstance(entry, str):
        return pd.NA
    try:
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and len(parsed) > 0:
            first = parsed[0]
            if isinstance(first, str):
                cleaned = first.strip()
                if cleaned.isdigit():
                    return int(cleaned)
                else:
                    return pd.NA
    except (ValueError, SyntaxError):
        pass
    return pd.NA

scout_car['Nr. of Doors'] = scout_car['Nr. of Doors'].apply(extract_num)
scout_car.rename(columns={'Nr. of Doors': '# of Doors'}, inplace=True)
scout_car['Nr. of Seats'] = scout_car['Nr. of Seats'].apply(extract_num)
scout_car.rename(columns={'Nr. of Seats': '# of Seats'}, inplace=True)

'Model Code' can be extracted by re-using an old function.

In [22]:
scout_car['Model Code'] = scout_car['Model Code'].apply(clean_offer_number)

'Gearing type' will also be easy.

In [23]:
scout_car['Gearing Type'] = scout_car['Gearing Type'].apply(extract_first_nonempty_str)

With 'Displacement' we will have to be a bit careful. This column renders numbers with a comma. First we will have to extract the number then convert it to numeric.

In [24]:
import ast
import re
import pandas as pd

def extract_displacement(entry):
    if not isinstance(entry, str):
        return pd.NA
    try:
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and len(parsed) > 0:
            first = parsed[0]
            if isinstance(first, str):
                # Remove commas and extract digits
                digits_only = re.sub(r'[^\d]', '', first)
                if digits_only.isdigit():
                    return int(digits_only)
    except (ValueError, SyntaxError):
        pass
    return pd.NA

scout_car['Displacement'] = scout_car['Displacement'].apply(extract_displacement)
scout_car.rename(columns={'Displacement': 'Displacement (cc)'}, inplace=True)

'Cylinders' is easier.

In [25]:
scout_car['Cylinders'] = scout_car['Cylinders'].apply(extract_num)

'Weight' goes back to the comma extract problem.

In [26]:
scout_car['Weight'] = scout_car['Weight'].apply(extract_displacement)
scout_car.rename(columns={'Weight': 'Weight (kg)'}, inplace=True)

'Drive chain' and 'Fuel' can re-use an old function.

In [27]:
scout_car['Drive chain'] = scout_car['Drive chain'].apply(extract_first_nonempty_str)
scout_car['Fuel'] = scout_car['Fuel'].apply(extract_first_nonempty_str)

'Consumption' requires special considerations. It lists fuel consumption rates for comb, city, and country, each having different values. We will split these into three columns.

In [28]:
# Split 'Consumption' into three appropriate categories
import ast
import re
import pandas as pd

def split_consumption(entry):
    values = {
        'Consumption (L /100 km) comb': pd.NA,
        'Consumption (L /100 km) city': pd.NA,
        'Consumption (L /100 km) country': pd.NA
    }
    
    if not isinstance(entry, str):
        return values
    
    try:
        parsed = ast.literal_eval(entry)  # convert string to list
    except (ValueError, SyntaxError):
        return values
    
    if not isinstance(parsed, list):
        return values
    
    for item in parsed:
        if isinstance(item, list) and len(item) > 0 and isinstance(item[0], str):
            text = item[0]
            # Extract numeric part
            num_match = re.search(r'([\d.,]+)', text)
            # Identify which type it is
            if '(comb)' in text:
                col = 'Consumption (L /100 km) comb'
            elif '(city)' in text:
                col = 'Consumption (L /100 km) city'
            elif '(country)' in text:
                col = 'Consumption (L /100 km) country'
            else:
                continue

            if num_match:
                value = float(num_match.group(1).replace(',', '.'))
                values[col] = value
    
    return values

# Apply function and expand into new columns
consumption_split = scout_car['Consumption'].apply(split_consumption).apply(pd.Series)
scout_car = pd.concat([scout_car, consumption_split], axis=1)

# Drop original consumption column
scout_car.drop('Consumption', axis=1, inplace=True)

CO2 Emission

In [29]:
def extract_co2(entry):
    if not isinstance(entry, str):
        return pd.NA
    try:
        parsed = ast.literal_eval(entry)
        if isinstance(parsed, (list, tuple)) and parsed:
            first = parsed[0]
            if isinstance(first, str):
                match = re.search(r'(\d+)', first)
                if match:
                    return int(match.group(1))
    except (ValueError, SyntaxError):
        pass
    return pd.NA

scout_car['CO2 Emission'] = scout_car['CO2 Emission'].apply(extract_co2)

# Rename column for clarity
scout_car.rename(columns={'CO2 Emission': 'CO2 Emission (g CO2/km (comb))'}, inplace=True)

'Emission Class'

In [30]:
scout_car['Emission Class'] = scout_car['Emission Class'].apply(extract_first_nonempty_str)

# Four columns with lists of features.

“Comfort & Convenience”

“Entertainment & Media”

“Extras”

“Safety & Security”

These column names contain hidden whitespace. We will have to strip them.

In [31]:
scout_car.shape

(15919, 45)

In [32]:
scout_car.columns = scout_car.columns.str.strip()

For each of these four columns, this function will:
1. Scan through each row and compile a list of all unique features
2. Turn each of these features into its own column, and populate with a binary indicated whether each car has that feature.

In [33]:
# This function will scan all rows and compile new columns from the features
import pandas as pd
import ast

# Step 0: Clean column names (strip whitespace/newlines)
scout_car.columns = scout_car.columns.str.strip()

# Reset index to clean RangeIndex to avoid join misalignment
scout_car = scout_car.reset_index(drop=True)

def expand_features(df, col_name):
    def to_list(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except:
                return []
        elif isinstance(val, list):
            return val
        else:
            return []

    df[col_name] = df[col_name].apply(to_list)
    exploded = df[col_name].explode()
    features_presence = pd.crosstab(exploded.index, exploded).astype(int)
    return features_presence

cols_to_expand = ['Comfort & Convenience', 'Entertainment & Media', 'Extras', 'Safety & Security']

for col in cols_to_expand:
    features_df = expand_features(scout_car, col)
    scout_car = scout_car.join(features_df)
    
    # Fill NaNs with 0 after join
    scout_car[features_df.columns] = scout_car[features_df.columns].fillna(0)
    
    # Convert to int dtype explicitly
    scout_car[features_df.columns] = scout_car[features_df.columns].astype(int)

# Now scout_car has binary 0/1 integer columns for all features, no suffixes, no decimals

In [34]:
scout_car.shape

(15919, 139)

This added 96 new collumns.

Now we can drop the original columns.

In [35]:
scout_car.drop('Comfort & Convenience', axis=1, inplace=True)
scout_car.drop('Entertainment & Media', axis=1, inplace=True)
scout_car.drop('Extras', axis=1, inplace=True)
scout_car.drop('Safety & Security', axis=1, inplace=True)

# Save to CSV

In [36]:
scout_car.to_csv('Step 1 - Cleaned v1.csv', index=False)

In [42]:
print(scout_car['Next Inspection (YYYYMM)'].dtype)

object


In [33]:
#scout_car.to_csv('Validation csv 3.csv', index=False)

In [35]:
#scout_car.to_csv('Final_Rough.csv', index=False)

In [37]:
scout_car[['registration']]

,registration
0,01/2016
1,03/2017
2,02/2016
3,08/2016
4,05/2016
...,...
15914,-/-
15915,01/2019
15916,03/2019
15917,06/2019


In [40]:
scout_car['registration'].value_counts(dropna=False)

registration
-/-        1597
03/2018     695
02/2019     585
05/2018     572
03/2019     543
01/2019     541
04/2018     541
02/2018     539
03/2016     536
04/2016     532
06/2018     532
01/2018     511
04/2019     506
02/2016     472
03/2017     471
05/2016     459
06/2016     452
05/2019     440
06/2017     409
05/2017     404
07/2018     396
04/2017     380
01/2016     376
02/2017     368
01/2017     306
08/2018     285
06/2019     224
07/2017     215
11/2017     180
07/2016     176
10/2016     160
10/2017     154
09/2017     149
11/2016     142
09/2018     141
09/2016     141
12/2016     134
12/2017     123
08/2017     114
11/2018     110
12/2018     103
10/2018      97
08/2016      94
07/2019       6
09/2019       5
08/2019       1
11/2019       1
12/2019       1
Name: count, dtype: int64